### Search Engine With Tools And Agents
Try more LangChain 🤝 Streamlit Agent examples at [github.com/langchain-ai/streamlit-agent](https://github.com/langchain-ai/streamlit-agent).

#### LangChainDeprecationWarning: 
**LangChain agents** will continue to be supported, but it is recommended for new use cases to be built with **LangGraph**. LangGraph offers a more flexible and full-featured framework for building agents, including support for **tool-calling, persistence of state**, and **human-in-the-loop** workflows.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [10]:
import openai
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent
from langchain import hub
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

In [8]:
groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = os.environ['OPENROUTER_API_KEY']

### Using the inbuilt tools of wikipedia and ArXiv

In [4]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results = 1, doc_content_chars_max = 250)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)
wiki.name

'wikipedia'

In [5]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max = 250)
arxiv = ArxivQueryRun(api_wrapper = api_wrapper_arxiv)
print(arxiv.name)

arxiv


### Tools

In [6]:
tools = [wiki, arxiv]

### Custom RAG Tool

In [11]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
docs_splits = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector_db = FAISS.from_documents(docs_splits, OllamaEmbeddings(model = 'gemma2:2b'))
retriever = vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021C28A5CC10>, search_kwargs={})

In [12]:
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about Langsmith ")
retriever_tool.name

'langsmith-search'

### Tools update

In [13]:
tools = [wiki, arxiv, retriever_tool]

In [14]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'C:\\Users\\cypro\\anaconda3\\envs\\genai_lchain\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000021C034BA660>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021C

### Tools: LLM-->AgentExecutor
Run all this tools with Agents and LLM Models

In [15]:
llm = ChatGroq(model_name = "llama-3.1-8b-instant")

### Prompt Template

In [16]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Agents

In [17]:
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

### Agent Executer

In [18]:
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [19]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith - Docs by LangChainDocs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQDocs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubForumGet started with LangSmithCopy pageCopy pageLangSmith is a platform for building production-grade LLM applications. Monitor and evaluate your application, so you can ship quickly and with confidence.
LangSmith is framework agnostic — you can use it with or without LangChain’s open source frameworks langchain and langgraph.

Start tracingGain

{'input': 'Tell me about Langsmith',
 'output': "The information provided about LangSmith indicates that it is a platform for building production-grade LLM applications, allowing users to monitor and evaluate their applications for debugging and development. It supports various features such as tracing, evaluation, and prompt testing. LangSmith also offers a pricing plan with a free option, although specific details about the free plan are not mentioned.\n\nTo get more information about the free plan, I would recommend checking the official LangSmith website or documentation for more details on pricing and plans.\n\nIn addition, if you have any further questions or need clarification on any of the features or pricing plans, I would be happy to help.\n\nIf you would like to explore other resources related to LangSmith or LLM applications, I can suggest searching online for relevant articles, tutorials, or forums where developers and experts share their experiences and knowledge.\n\nPlea

In [21]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as
Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witIt seems that the provided arXiv result was not directly relevant to the question. I have now added an alternative result from Wikipedia that should be more relevant.

> Finished chain.


{'input': 'What is machine learning',
 'output': 'It seems that the provided arXiv result was not directly relevant to the question. I have now added an alternative result from Wikipedia that should be more relevant.'}

In [22]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `arxiv` with `{'query': '1706.03762'}`
responded: It seems the previous response was cut off and the response was incomplete. Here is a revised response.



Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Sum

{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper 1706.03762, titled "Attention Is All You Need", is about the dominant sequence transduction models being based on convolutional neural networks (CNNs) and recurrent neural networks (RNNs). The authors propose a new model that uses self-attention mechanisms to compute representations of input sequences without relying on convolutions or recurrent operations. This model, called the Transformer, is capable of handling long-range dependencies and is particularly well-suited for tasks such as machine translation, text summarization, and question answering. The paper presents experiments demonstrating the effectiveness of the Transformer model on several benchmark datasets, and provides a detailed analysis of its architecture and training procedures.'}

## Using the `Client` API to pull prompt

In [23]:
from langsmith import Client
client = Client()
prompt = client.pull_prompt("hwchase17/openai-functions-agent", include_model=True)
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)

In [25]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith - Docs by LangChainDocs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQDocs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubForumGet started with LangSmithCopy pageCopy pageLangSmith is a platform for building production-grade LLM applications. Monitor and evaluate your application, so you can ship quickly and with confidence.
LangSmith is framework agnostic — you can use it with or without LangChain’s open source frameworks langchain and langgraph.

Start tracingGain

{'input': 'Tell me about Langsmith',
 'output': "I was unable to verify the accuracy of the following text:\n\n Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of languag"}

In [26]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cIt seems the paper 1706.03762 is actually "Attention Is All You Need" by Vaswani et al.

> Finished chain.


{'input': "What's the paper 1706.03762 about?",
 'output': 'It seems the paper 1706.03762 is actually "Attention Is All You Need" by Vaswani et al.'}

In [27]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as
Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witIt appears that the provided functions only return a summary of the top result, not the full Wikipedia page. Therefore, the full Wikipedia page for "machine learning" cannot be retrieved with the provided functions.

> Finished chain.


{'input': 'What is machine learning',
 'output': 'It appears that the provided functions only return a summary of the top result, not the full Wikipedia page. Therefore, the full Wikipedia page for "machine learning" cannot be retrieved with the provided functions.'}

## Using the tools with default parameters

In [28]:
api_wrapper_wiki = WikipediaAPIWrapper()
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)

api_wrapper_arxiv = ArxivAPIWrapper()
arxiv = ArxivQueryRun(api_wrapper = api_wrapper_arxiv)

tools = [wiki, arxiv, retriever_tool]
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)

### Run the agents

In [29]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith - Docs by LangChainDocs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQDocs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubForumGet started with LangSmithCopy pageCopy pageLangSmith is a platform for building production-grade LLM applications. Monitor and evaluate your application, so you can ship quickly and with confidence.
LangSmith is framework agnostic — you can use it with or without LangChain’s open source frameworks langchain and langgraph.

Start tracingGain

{'input': 'Tell me about Langsmith',
 'output': "The Langsmith platform offers a flexible pricing plan, with options for both free and paid tiers. The free tier includes access to all features, while the paid tier offers additional features and support. The pricing plan is as follows:\n\n* Free tier: $0/month\n* Paid tier: $50/user/month (billed annually)\n\nNote: The pricing plan is subject to change and may not be up-to-date. It's best to check the Langsmith website for the most current pricing information."}

In [30]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, based
solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to be
superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014
English-to-German translation task, improving over the existing best results,
including ensembles by over 

{'input': "What's the paper 1706.03762 about?",
 'output': "It seems that the provided paper '1706.03762' is not related to LangSmith. The paper '1706.03762' is actually the paper that introduced the Transformer model, which is an attention-based model for sequence-to-sequence tasks."}

In [31]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as well as
Simons Foundation, Berkeley.

Published: 2018-11-11
Title: An Optimal Control View of Adversarial Machine Learning
Authors: Xiaojin Zhu
Summary: I describe an optimal control view of adversarial machine learning, where the
dynamical system is the machine learner, the input are adversarial actions, and
the control costs are defined by the adversary's goals to do harm and be hard
to detect. This view encompasses many types of adversarial machine learning,
including test-item attacks, training-data poisoning, and adversarial reward
shaping. The view encourages adversarial machine learning researcher to utilize
advances in control theory a

{'input': 'What is machine learning',
 'output': 'I have been asked to assist with a machine learning question and I have provided multiple sources of information regarding the topic, including arXiv papers and Wikipedia articles.'}